In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI SDK：使用BigQuery为在线预测训练表格回归模型

<table align="left">
  <td>
<a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/automl/sdk_automl_tabular_regression_online_bq.ipynb" target='_blank'>
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
<a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/automl/sdk_automl_tabular_regression_online_bq.ipynb" target='_blank'>      
    <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/automl/sdk_automl_tabular_regression_online_bq.ipynb">
        <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在Vertex AI Workbench中打开
    </a>
  </td>
</table>
<br/><br/><br/>

## 概述

本教程演示了如何使用Python的Vertex AI SDK创建表格回归模型，并使用Google Cloud的[AutoML](https://cloud.google.com/vertex-ai/docs/start/automl-users)模型进行在线预测。

了解更多有关[表格数据的回归](https://cloud.google.com/vertex-ai/docs/tabular-data/classification-regression/overview)的信息。

### 目标

在本教程中，您将学习如何使用Vertex AI SDK从Python脚本创建一个AutoML表格回归模型，并将其部署用于在线预测。您还可以选择使用`gcloud`命令行工具或通过Cloud Console在线创建和部署模型。

执行的步骤包括：

- 创建一个Vertex `数据集`资源。
- 训练模型。
- 查看模型评估。
- 部署`模型`资源到一个服务`端点`资源。
- 进行预测。
- 取消部署`模型`。

### 数据集

本教程使用的数据集是来自[BigQuery公共数据集](https://cloud.google.com/bigquery/public-data)的GSOD数据集。您所使用的数据集版本仅包括年、月和日字段，以预测平均每日温度（mean_temp）的值。

### 成本

本教程使用 Google Cloud 的计费组件：

* Vertex AI
* 云存储

了解 [Vertex AI
定价](https://cloud.google.com/vertex-ai/pricing) 和 [云存储
定价](https://cloud.google.com/storage/pricing)，并使用 [定价计算器](https://cloud.google.com/products/calculator/)
根据您的预期使用情况生成成本估算。

## 安装

安装最新版本的 Python 用于 Vertex AI SDK。

In [ ]:
! pip3 install --upgrade google-cloud-aiplatform \
                            google-cloud-storage

### 仅限协作：取消下面的单元格注释以重新启动内核

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

## 开始之前

### GPU 运行时

本教程不需要 GPU 运行时。

### 设置您的 Google Cloud 项目

**如果您不知道您的项目 ID，** 可以尝试以下操作：

- 运行 `gcloud config list`
- 运行 `gcloud projects list`
- 查看支持页面：[定位项目 ID](https://support.google.com/googleapi/answer/7014113)

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project ID
! gcloud config set project {PROJECT_ID}

#### 区域

您也可以更改 Vertex AI 使用的 `REGION` 变量。了解有关 [Vertex AI 区域](https://cloud.google.com/vertex-ai/docs/general/locations) 的更多信息。

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

### 验证您的谷歌云帐户

根据您的Jupyter环境，您可能需要手动进行身份验证。请按照以下相关说明操作。

**1. Vertex AI workbench**
- 无需进行任何操作，因为您已经通过身份验证。

**2. 本地 JupyterLab 实例，请取消注释并运行：**

In [ ]:
# ! gcloud auth login

3. 合作，取消注释并运行：

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

服务帐号或其他- 在此处查看所有身份验证选项：[Google Cloud Platform Jupyter Notebook Authentication Guide](https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/notebook_authentication_guide.ipynb)

创建云存储桶
创建一个存储桶来存储中间工件，如数据集。

In [ ]:
BUCKET_URI = "gs://test-bucket-unique"  # @param {type:"string"}

只有在您的存储桶不存在时：运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_URI

导入库并定义常量

In [ ]:
import os

import google.cloud.aiplatform as aiplatform

display_name = "gsod_unique"

初始化顶点 AI SDK 用于 Python

为您的项目和相应的存储桶初始化顶点 AI SDK 用于 Python。

In [ ]:
aiplatform.init(project=PROJECT_ID, location=REGION)

教程

现在您已经准备好开始创建您自己的AutoML表格回归模型。

#### BigQuery培训数据的位置。

现在将变量`IMPORT_FILE`设置为BigQuery中数据表的位置。

In [ ]:
IMPORT_FILE = "bq://bigquery-public-data.samples.gsod"

### 创建数据集

接下来，使用`TabularDataset`类的`create`方法创建`Dataset`资源，需要传入以下参数：

- `display_name`：`Dataset`资源的人类可读名称。
- `bq_source`：或者，从BigQuery表导入数据项到`Dataset`资源。

这个操作可能需要几分钟时间。

In [ ]:
dataset = aiplatform.TabularDataset.create(
    display_name="NOAA historical weather data_unique",
    bq_source=[IMPORT_FILE],
)

label_column = "mean_temp"

print(dataset.resource_name)

In [ ]:
TRANSFORMATIONS = [
    {"auto": {"column_name": "year"}},
    {"auto": {"column_name": "month"}},
    {"auto": {"column_name": "day"}},
]

label_column = "mean_temp"

### 创建并运行训练流水线

要训练一个AutoML模型，您需要执行两个步骤：1）创建一个训练流水线，和2）运行这个流水线。

#### 创建训练流水线

使用`AutoMLTabularTrainingJob`类创建一个AutoML训练流水线，并设置以下参数：

- `display_name`：`TrainingJob`资源的可读性名称。
- `optimization_prediction_type`：训练模型的任务类型。
  - `classification`：表格分类模型。
  - `regression`：表格回归模型。
- `column_transformations`：（可选）要应用于输入列的转换。
- `optimization_objective`：要最小化或最大化的优化目标。
  - 二元分类：
    - `minimize-log-loss`
    - `maximize-au-roc`
    - `maximize-au-prc`
    - `maximize-precision-at-recall`
    - `maximize-recall-at-precision`
  - 多类分类：
    - `minimize-log-loss`
  - 回归：
    - `minimize-rmse`
    - `minimize-mae`
    - `minimize-rmsle`

实例化的对象是训练流水线的有向无环图（DAG）。

In [ ]:
job = aiplatform.AutoMLTabularTrainingJob(
    display_name=display_name,
    optimization_prediction_type="regression",
    optimization_objective="minimize-rmse",
    column_transformations=TRANSFORMATIONS,
)

print(job)

#### 运行训练流水线

接下来，您可以运行DAG（对象'job'）来启动训练作业，通过调用`run`方法并传入以下参数：

- `dataset`：用于训练模型的`Dataset`资源。
- `model_display_name`：训练模型的可读名称。
- `training_fraction_split`：用于训练的数据集百分比。
- `test_fraction_split`：用于测试（留出数据）的数据集百分比。
- `validation_fraction_split`：用于验证的数据集百分比。
- `target_column`：要作为标签进行训练的列名。
- `budget_milli_node_hours`：（可选）以毫小时为单位指定的最大训练时间（1000 = 小时）。
- `disable_early_stopping`：如果设置为`True`，则服务可能在认为无法进一步改进模型目标测量之前在整个预算范围内完成训练。

完成`run`方法后，将返回`Model`资源。

此训练流水线的执行将需要最多8小时。

In [ ]:
model = job.run(
    dataset=dataset,
    model_display_name=display_name,
    training_fraction_split=0.6,
    validation_fraction_split=0.2,
    test_fraction_split=0.2,
    budget_milli_node_hours=8000,
    disable_early_stopping=False,
    target_column=label_column,
)

## 查看模型评估分数
在您的模型训练完成后，您可以使用 list_model_evaluations() 方法查看其评估分数。

In [ ]:
model_evaluations = model.list_model_evaluations()
if len(model_evaluations) > 0:
    eval_res = model_evaluations[0].to_dict()
    evaluation_metrics = eval_res["metrics"]
print(evaluation_metrics)

部署模型

接下来，部署您的模型以进行在线预测。要部署模型，您需要调用`deploy` 方法，并传入以下参数：

- `machine_type`：计算机类型。

In [ ]:
endpoint = model.deploy(machine_type="n1-standard-4")

发送一个在线预测请求

向已部署的模型发送在线预测。

创建测试项目

您将使用合成数据作为测试数据项目。请不必担心我们使用的是合成数据 - 我们只是想演示如何进行预测。

In [ ]:
INSTANCE = {"year": "1932", "month": "11", "day": "6"}

### 进行预测

现在您的`Model`资源已部署到一个`Endpoint`资源，您可以通过向`Endpoint`资源发送预测请求来进行在线预测。

#### 请求

每个实例的格式为：

    [feature_list]

由于`predict()`方法可以接受多个项目（实例），请将您的单个测试项目作为一个测试项目列表发送。

#### 响应

从`predict()`调用的响应是一个具有以下条目的Python字典：

- `ids`：每个预测请求的内部分配的唯一标识符。
- `value`：每个预测的预测值。
- `deployed_model_id`：进行预测的已部署`Model`资源的Vertex AI标识符。

In [ ]:
instances_list = [INSTANCE]

prediction = endpoint.predict(instances_list)
print(prediction)

解除部署模型

当你完成预测时，你可以从 `Endpoint` 资源中解除部署模型。这将取消所有计算资源，并停止部署模型时的计费。

In [ ]:
endpoint.undeploy_all()

清理工作

要清理本项目中使用的所有Google Cloud资源，您可以[删除用于本教程的Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除本教程中创建的个别资源：

- 数据集
- 模型
- 端点
- AutoML训练作业

In [ ]:
# Delete the dataset using the Vertex dataset object
dataset.delete()

# Delete the model using the Vertex model object
model.delete()

# Delete the endpoint using the Vertex endpoint object
endpoint.delete()

# Delete the AutoML trainig job
job.delete()

delete_bucket = False

if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -r $BUCKET_URI